# LSIによる次元圧縮の動作確認

ストーリー: https://www.pivotaltracker.com/story/show/149341617

LSI_in_sckit_learn.ipynbの続きで、他の質問などを実験した結果を収録

In [1]:
import sys
import os

os.getcwd()

'/Users/shwld/project/mofmof/donusagi-bot/learning/prototype'

In [2]:
learning_dir = os.path.abspath("../")
os.chdir(learning_dir)

if learning_dir not in sys.path:
    sys.path.append(learning_dir)

In [3]:
from learning.core.nlang import Nlang
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

## データの準備

In [4]:
df = pd.read_csv('./prototype/fixtures/learning_training_messages/LSI_in_sckit_learn.csv')
sentences = np.array(df['question'])
separated_sentences = Nlang.batch_split(sentences)
separated_sentences

['育児 短時間 勤務 申請 書 いつ 提出 する',
 '育児 短時間 勤務 申請 書 曜日 別 申請 できる',
 '育児 短時間 勤務 申請 どこ 提出 する よい',
 '子供 生まれる',
 'ＶＩＳＡ 勘定 科目 わかる ない',
 '海外 出張 する 旅費 精算 する どう する 良い',
 '保険証 失 くい']

## TfidfVectorizerを用いてベクトルを作る

In [6]:
vectorizer = TfidfVectorizer(use_idf=True, token_pattern=u'(?u)\\b\\w+\\b')
features = vectorizer.fit_transform(separated_sentences)

## 検索する単語も同様に処理する

In [7]:
target_sentences = np.array(['保険証をなくした'])
target_separated_sentences = Nlang.batch_split(target_sentences)
target_features = vectorizer.transform(target_separated_sentences)
print(target_separated_sentences)
target_sentences2 = np.array(['保険証を失くした'])
target_separated_sentences2 = Nlang.batch_split(target_sentences2)
target_features2 = vectorizer.transform(target_separated_sentences2)
target_separated_sentences2

['保険証 なくす']


['保険証 失 くい']

## コサイン類似検索をかけてみる(LSIなし)

In [8]:
similarities = cosine_similarity(features, target_features)
similarities = similarities.flatten()
zipped_data = zip(similarities, separated_sentences, np.array(df['question']))
list(sorted(zipped_data, key=lambda x: x[0], reverse=True))[0:10]

[(0.57735026918962573, '保険証 失 くい', '保険証を失くした'),
 (0.0, '育児 短時間 勤務 申請 書 いつ 提出 する', '育児短時間勤務申請書はいつまでに提出するのか？'),
 (0.0, '育児 短時間 勤務 申請 書 曜日 別 申請 できる', '育児短時間勤務申請書は曜日別に申請できるか？'),
 (0.0, '育児 短時間 勤務 申請 どこ 提出 する よい', '育児短時間勤務申請はどこへ提出すればよいか？'),
 (0.0, '子供 生まれる', '子供が生まれた'),
 (0.0, 'ＶＩＳＡ 勘定 科目 わかる ない', 'VISAの勘定科目がわからない'),
 (0.0, '海外 出張 する 旅費 精算 する どう する 良い', '海外出張したときの旅費精算をしたいがどうしたら良い')]

In [9]:
similarities = cosine_similarity(features, target_features2)
similarities = similarities.flatten()
zipped_data = zip(similarities, separated_sentences, np.array(df['question']))
list(sorted(zipped_data, key=lambda x: x[0], reverse=True))[0:10]

[(1.0, '保険証 失 くい', '保険証を失くした'),
 (0.0, '育児 短時間 勤務 申請 書 いつ 提出 する', '育児短時間勤務申請書はいつまでに提出するのか？'),
 (0.0, '育児 短時間 勤務 申請 書 曜日 別 申請 できる', '育児短時間勤務申請書は曜日別に申請できるか？'),
 (0.0, '育児 短時間 勤務 申請 どこ 提出 する よい', '育児短時間勤務申請はどこへ提出すればよいか？'),
 (0.0, '子供 生まれる', '子供が生まれた'),
 (0.0, 'ＶＩＳＡ 勘定 科目 わかる ない', 'VISAの勘定科目がわからない'),
 (0.0, '海外 出張 する 旅費 精算 する どう する 良い', '海外出張したときの旅費精算をしたいがどうしたら良い')]

## TruncatedSVDで次元圧縮する

In [10]:
lsa = TruncatedSVD(n_components=6, algorithm='randomized', n_iter=10, random_state=42)
lsa_features = lsa.fit_transform(features)
normalizer = Normalizer(copy=False)
lsa_features = normalizer.fit_transform(lsa_features)

In [11]:
lsa_target_features = lsa.transform(target_features)
lsa_target_features = normalizer.transform(lsa_target_features)
lsa_target_features2 = lsa.transform(target_features2)
lsa_target_features2 = normalizer.transform(lsa_target_features2)

In [12]:
# LSI無し
print(features.shape)
print(target_features.shape)
print(target_features2.shape)

# LSIあり
print(lsa_features.shape)
print(lsa_target_features.shape)
print(lsa_target_features2.shape)

(7, 29)
(1, 29)
(1, 29)
(7, 6)
(1, 6)
(1, 6)


## コサイン類似検索をかけてみる(LSIあり)

In [13]:
lsa_similarities = cosine_similarity(lsa_features, lsa_target_features)
lsa_similarities = lsa_similarities.flatten()
zipped_data = zip(lsa_similarities, separated_sentences, np.array(df['question']))
list(sorted(zipped_data, key=lambda x: x[0], reverse=True))[:10]

[(1.0, '保険証 失 くい', '保険証を失くした'),
 (2.5210675458026379e-16,
  '海外 出張 する 旅費 精算 する どう する 良い',
  '海外出張したときの旅費精算をしたいがどうしたら良い'),
 (1.1449174941446927e-16, '子供 生まれる', '子供が生まれた'),
 (4.292784406596532e-17, '育児 短時間 勤務 申請 書 いつ 提出 する', '育児短時間勤務申請書はいつまでに提出するのか？'),
 (6.797588530016039e-18, '育児 短時間 勤務 申請 どこ 提出 する よい', '育児短時間勤務申請はどこへ提出すればよいか？'),
 (0.0, 'ＶＩＳＡ 勘定 科目 わかる ない', 'VISAの勘定科目がわからない'),
 (-1.3713376785647671e-16,
  '育児 短時間 勤務 申請 書 曜日 別 申請 できる',
  '育児短時間勤務申請書は曜日別に申請できるか？')]

In [14]:
lsa_similarities = cosine_similarity(lsa_features, lsa_target_features2)
lsa_similarities = lsa_similarities.flatten()
zipped_data = zip(lsa_similarities, separated_sentences, np.array(df['question']))
list(sorted(zipped_data, key=lambda x: x[0], reverse=True))[:10]

[(0.99999999999999989, '保険証 失 くい', '保険証を失くした'),
 (1.8626326654009679e-16,
  '海外 出張 する 旅費 精算 する どう する 良い',
  '海外出張したときの旅費精算をしたいがどうしたら良い'),
 (1.457167719820518e-16, '子供 生まれる', '子供が生まれた'),
 (8.3266726846886741e-17, 'ＶＩＳＡ 勘定 科目 わかる ない', 'VISAの勘定科目がわからない'),
 (4.7223693784585185e-17,
  '育児 短時間 勤務 申請 書 いつ 提出 する',
  '育児短時間勤務申請書はいつまでに提出するのか？'),
 (1.3129907805395033e-17,
  '育児 短時間 勤務 申請 どこ 提出 する よい',
  '育児短時間勤務申請はどこへ提出すればよいか？'),
 (-1.1942777452914207e-16,
  '育児 短時間 勤務 申請 書 曜日 別 申請 できる',
  '育児短時間勤務申請書は曜日別に申請できるか？')]

# LDAも試してみる

試せそうだったので `LDA_in_scikit_learn_xx.ipynb` あたりを参考にLDAを試して比較してみた。

http://hivecolor.com/id/65

によると、

> LSIもLDAも、次元をトピック単位に縮約するのは同じでした。lsiコーパスとldaコーパスの値もほとんど同じです。ただ、全く同じではなく、LSIとLDAには、「確率的な揺らぎを考慮するかどうか」という違いがあります。

> LSIでは、一緒に使われている単語であれば、そのうち片方の単語で検索すると、もう片方の単語の結果も見付かることをちょっと前に書きました。しかしながら、どれだけ意味が似ている単語であっても、一緒に使われていなければそういう検索はできないという弱点があります。

> LDAでは、LSIに確率分布を取り入れることで、直接的には一緒に使われていない単語同士であっても検索で見付けることができるようになります。

> これまた詳しくないので詳細な説明はできないのですが、利用者の立場としては、LSIの発展版がLDAだ、くらいに思っておけばいいんじゃないかと思います。

のように書いてあります。

In [15]:
from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(n_topics=8, 
                                max_iter=5,
                                learning_method='online')

lda_features = lda.fit_transform(features)
lda_features = normalizer.fit_transform(lda_features)

In [16]:
lda_target_features = lda.transform(target_features)
lda_target_features = normalizer.transform(lda_target_features)
lda_target_features2 = lda.transform(target_features2)
lda_target_features2 = normalizer.transform(lda_target_features2)

In [17]:
# LDA無し
print(features.shape)
print(target_features.shape)
print(target_features2.shape)

# LDAあり
print(lda_features.shape)
print(lda_target_features.shape)
print(lda_target_features2.shape)

(7, 29)
(1, 29)
(1, 29)
(7, 8)
(1, 8)
(1, 8)


## コサイン類似検索をかけてみる(LDAあり)

In [18]:
lda_similarities = cosine_similarity(lda_features, lda_target_features)
lda_similarities = lda_similarities.flatten()
zipped_data = zip(lda_similarities, separated_sentences, np.array(df['question']))
list(sorted(zipped_data, key=lambda x: x[0], reverse=True))[:10]

[(0.99396445489149177, '保険証 失 くい', '保険証を失くした'),
 (0.23150324401538841, '子供 生まれる', '子供が生まれた'),
 (0.18967054053455626, 'ＶＩＳＡ 勘定 科目 わかる ない', 'VISAの勘定科目がわからない'),
 (0.18146550140387133,
  '海外 出張 する 旅費 精算 する どう する 良い',
  '海外出張したときの旅費精算をしたいがどうしたら良い'),
 (0.1753443110691506, '育児 短時間 勤務 申請 書 曜日 別 申請 できる', '育児短時間勤務申請書は曜日別に申請できるか？'),
 (0.17418927440477222, '育児 短時間 勤務 申請 どこ 提出 する よい', '育児短時間勤務申請はどこへ提出すればよいか？'),
 (0.17394767291071614, '育児 短時間 勤務 申請 書 いつ 提出 する', '育児短時間勤務申請書はいつまでに提出するのか？')]

## コサイン類似検索をかけてみる(LDAあり)

In [19]:
lda_similarities = cosine_similarity(lda_features, lda_target_features2)
lda_similarities = lda_similarities.flatten()
zipped_data = zip(lda_similarities, separated_sentences, np.array(df['question']))
list(sorted(zipped_data, key=lambda x: x[0], reverse=True))[:10]

[(1.0000000000000002, '保険証 失 くい', '保険証を失くした'),
 (0.17471733435520553, '子供 生まれる', '子供が生まれた'),
 (0.1382477701762275, 'ＶＩＳＡ 勘定 科目 わかる ない', 'VISAの勘定科目がわからない'),
 (0.1310932281165654,
  '海外 出張 する 旅費 精算 する どう する 良い',
  '海外出張したときの旅費精算をしたいがどうしたら良い'),
 (0.12575668637276424, '育児 短時間 勤務 申請 書 曜日 別 申請 できる', '育児短時間勤務申請書は曜日別に申請できるか？'),
 (0.12476713970111256, '育児 短時間 勤務 申請 どこ 提出 する よい', '育児短時間勤務申請はどこへ提出すればよいか？'),
 (0.12455669954235238, '育児 短時間 勤務 申請 書 いつ 提出 する', '育児短時間勤務申請書はいつまでに提出するのか？')]

パラメータ調整
http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html
を参考にパラメータを調整してみた。

何か間違っているのかもしれないが、色々調整してみても、上記のように全く予測できていない。

# 結果まとめ

## 保険証なくした

similalityが向上している。
次元削減によりあまり関係のない特徴が無視され、精度が上がったものと思われる。
削減後の次元数を少なくするほどsimilalityは向上するのでちょうどいい削減数にしたい。

既存

```
[(0.36489794292209582, '保険証 失 くい', '保険証を失くした'),
 (0.29414540592718241,
  '海外 出張 する 旅費 精算 する どう する 良い',
  '海外出張したときの旅費精算をしたいがどうしたら良い'),
 (0.28264873120052436, 'ＶＩＳＡ 勘定 科目 わかる ない', 'VISAの勘定科目がわからない'),
 (0.1438089997699937, '育児 短時間 勤務 申請 書 いつ 提出 する', '育児短時間勤務申請書はいつまでに提出するのか？'),
 (0.13944815278566233, '育児 短時間 勤務 申請 どこ 提出 する よい', '育児短時間勤務申請はどこへ提出すればよいか？'),
 (0.0, '育児 短時間 勤務 申請 書 曜日 別 申請 できる', '育児短時間勤務申請書は曜日別に申請できるか？'),
 (0.0, '子供 生まれる', '子供が生まれた')]
```

LSI

```
[(0.65367129716499184, '保険証 失 くい', '保険証を失くした'),
 (0.52692653597281136,
  '海外 出張 する 旅費 精算 する どう する 良い',
  '海外出張したときの旅費精算をしたいがどうしたら良い'),
 (0.50633160956276302, 'ＶＩＳＡ 勘定 科目 わかる ない', 'VISAの勘定科目がわからない'),
 (0.25761673160136522, '育児 短時間 勤務 申請 書 いつ 提出 する', '育児短時間勤務申請書はいつまでに提出するのか？'),
 (0.24980479250913942, '育児 短時間 勤務 申請 どこ 提出 する よい', '育児短時間勤務申請はどこへ提出すればよいか？'),
 (8.4879361989241347e-17, '子供 生まれる', '子供が生まれた'),
 (-2.4449837541879543e-16,
  '育児 短時間 勤務 申請 書 曜日 別 申請 できる',
  '育児短時間勤務申請書は曜日別に申請できるか？')]
```

LDA

```
[(0.99999763042976986, '保険証 失 くい', '保険証を失くした'),
 (0.99922841796653905, 'ＶＩＳＡ 勘定 科目 わかる ない', 'VISAの勘定科目がわからない'),
 (0.17583143931504724, '子供 生まれる', '子供が生まれた'),
 (0.13217406909552246,
  '海外 出張 する 旅費 精算 する どう する 良い',
  '海外出張したときの旅費精算をしたいがどうしたら良い'),
 (0.12670173010146185, '育児 短時間 勤務 申請 書 曜日 別 申請 できる', '育児短時間勤務申請書は曜日別に申請できるか？'),
 (0.1258307199815161, '育児 短時間 勤務 申請 どこ 提出 する よい', '育児短時間勤務申請はどこへ提出すればよいか？'),
 (0.12562963377518871, '育児 短時間 勤務 申請 書 いつ 提出 する', '育児短時間勤務申請書はいつまでに提出するのか？')]
```

## 保険証失くした

既存

```
[(1.0, '保険証 失 くい', '保険証を失くした'),
 (0.0, '育児 短時間 勤務 申請 書 いつ 提出 する', '育児短時間勤務申請書はいつまでに提出するのか？'),
 (0.0, '育児 短時間 勤務 申請 書 曜日 別 申請 できる', '育児短時間勤務申請書は曜日別に申請できるか？'),
 (0.0, '育児 短時間 勤務 申請 どこ 提出 する よい', '育児短時間勤務申請はどこへ提出すればよいか？'),
 (0.0, '子供 生まれる', '子供が生まれた'),
 (0.0, 'ＶＩＳＡ 勘定 科目 わかる ない', 'VISAの勘定科目がわからない'),
 (0.0, '海外 出張 する 旅費 精算 する どう する 良い', '海外出張したときの旅費精算をしたいがどうしたら良い')]
```

LSI

```
[(1.0000000000000002, '保険証 失 くい', '保険証を失くした'),
 (1.6653345369377348e-16, '子供 生まれる', '子供が生まれた'),
 (9.8088920827404037e-17,
  '育児 短時間 勤務 申請 どこ 提出 する よい',
  '育児短時間勤務申請はどこへ提出すればよいか？'),
 (-7.8682544391720894e-18,
  '育児 短時間 勤務 申請 書 いつ 提出 する',
  '育児短時間勤務申請書はいつまでに提出するのか？'),
 (-5.5747247630078562e-17,
  '育児 短時間 勤務 申請 書 曜日 別 申請 できる',
  '育児短時間勤務申請書は曜日別に申請できるか？'),
 (-5.8754446610958302e-17, 'ＶＩＳＡ 勘定 科目 わかる ない', 'VISAの勘定科目がわからない'),
 (-1.099234254312998e-16,
  '海外 出張 する 旅費 精算 する どう する 良い',
  '海外出張したときの旅費精算をしたいがどうしたら良い')]
```

LDA

```
[(1.0000000000000002, '保険証 失 くい', '保険証を失くした'),
 (0.99931138558358357, 'ＶＩＳＡ 勘定 科目 わかる ない', 'VISAの勘定科目がわからない'),
 (0.17472768450420262, '子供 生まれる', '子供が生まれた'),
 (0.1310833671368809,
  '海外 出張 する 旅費 精算 する どう する 良い',
  '海外出張したときの旅費精算をしたいがどうしたら良い'),
 (0.12574492003007595, '育児 短時間 勤務 申請 書 曜日 別 申請 できる', '育児短時間勤務申請書は曜日別に申請できるか？'),
 (0.12475810534454906, '育児 短時間 勤務 申請 どこ 提出 する よい', '育児短時間勤務申請はどこへ提出すればよいか？'),
 (0.12455763825637048, '育児 短時間 勤務 申請 書 いつ 提出 する', '育児短時間勤務申請書はいつまでに提出するのか？')]
```


## こどもがうまれた

どの単語もボキャブラリーにないため、回答できない。
LDAの場合は確率分布により予測できている。

既存

```
[(0.0, '育児 短時間 勤務 申請 書 いつ 提出 する', '育児短時間勤務申請書はいつまでに提出するのか？'),
 (0.0, '育児 短時間 勤務 申請 書 曜日 別 申請 できる', '育児短時間勤務申請書は曜日別に申請できるか？'),
 (0.0, '育児 短時間 勤務 申請 どこ 提出 する よい', '育児短時間勤務申請はどこへ提出すればよいか？'),
 (0.0, '子供 生まれる', '子供が生まれた'),
 (0.0, 'ＶＩＳＡ 勘定 科目 わかる ない', 'VISAの勘定科目がわからない'),
 (0.0, '海外 出張 する 旅費 精算 する どう する 良い', '海外出張したときの旅費精算をしたいがどうしたら良い'),
 (0.0, '保険証 失 くい', '保険証を失くした')]
```

LSI

```
[(0.0, '育児 短時間 勤務 申請 書 いつ 提出 する', '育児短時間勤務申請書はいつまでに提出するのか？'),
 (0.0, '育児 短時間 勤務 申請 書 曜日 別 申請 できる', '育児短時間勤務申請書は曜日別に申請できるか？'),
 (0.0, '育児 短時間 勤務 申請 どこ 提出 する よい', '育児短時間勤務申請はどこへ提出すればよいか？'),
 (0.0, '子供 生まれる', '子供が生まれた'),
 (0.0, 'ＶＩＳＡ 勘定 科目 わかる ない', 'VISAの勘定科目がわからない'),
 (0.0, '海外 出張 する 旅費 精算 する どう する 良い', '海外出張したときの旅費精算をしたいがどうしたら良い'),
 (0.0, '保険証 失 くい', '保険証を失くした')]
```

LDA

```
[(0.54232500732829159, '子供 生まれる', '子供が生まれた'),
 (0.51226155701256915, '保険証 失 くい', '保険証を失くした'),
 (0.48004031868096553, 'ＶＩＳＡ 勘定 科目 わかる ない', 'VISAの勘定科目がわからない'),
 (0.46766797607224725,
  '海外 出張 する 旅費 精算 する どう する 良い',
  '海外出張したときの旅費精算をしたいがどうしたら良い'),
 (0.45838192185609988, '育児 短時間 勤務 申請 書 曜日 別 申請 できる', '育児短時間勤務申請書は曜日別に申請できるか？'),
 (0.45673056830455627, '育児 短時間 勤務 申請 どこ 提出 する よい', '育児短時間勤務申請はどこへ提出すればよいか？'),
 (0.45635712584466431, '育児 短時間 勤務 申請 書 いつ 提出 する', '育児短時間勤務申請書はいつまでに提出するのか？')]
```

## こどもが生まれた

既存

```
[(0.70710678118654757, '子供 生まれる', '子供が生まれた'),
 (0.0, '育児 短時間 勤務 申請 書 いつ 提出 する', '育児短時間勤務申請書はいつまでに提出するのか？'),
 (0.0, '育児 短時間 勤務 申請 書 曜日 別 申請 できる', '育児短時間勤務申請書は曜日別に申請できるか？'),
 (0.0, '育児 短時間 勤務 申請 どこ 提出 する よい', '育児短時間勤務申請はどこへ提出すればよいか？'),
 (0.0, 'ＶＩＳＡ 勘定 科目 わかる ない', 'VISAの勘定科目がわからない'),
 (0.0, '海外 出張 する 旅費 精算 する どう する 良い', '海外出張したときの旅費精算をしたいがどうしたら良い'),
 (0.0, '保険証 失 くい', '保険証を失くした')]
```

LSI

```
[(1.0, '子供 生まれる', '子供が生まれた'),
 (1.3972112760024048e-16,
  '育児 短時間 勤務 申請 書 曜日 別 申請 できる',
  '育児短時間勤務申請書は曜日別に申請できるか？'),
 (1.1102230246251565e-16, 'ＶＩＳＡ 勘定 科目 わかる ない', 'VISAの勘定科目がわからない'),
 (5.5511151231257827e-17, '保険証 失 くい', '保険証を失くした'),
 (1.4328446680903843e-17,
  '育児 短時間 勤務 申請 書 いつ 提出 する',
  '育児短時間勤務申請書はいつまでに提出するのか？'),
 (-5.3945024184152252e-17,
  '育児 短時間 勤務 申請 どこ 提出 する よい',
  '育児短時間勤務申請はどこへ提出すればよいか？'),
 (-1.3981097219342767e-16,
  '海外 出張 する 旅費 精算 する どう する 良い',
  '海外出張したときの旅費精算をしたいがどうしたら良い')]
```

LDA

```
[(0.99723020929412565, '子供 生まれる', '子供が生まれた'),
 (0.21120057656529864, '保険証 失 くい', '保険証を失くした'),
 (0.18963963728602987, 'ＶＩＳＡ 勘定 科目 わかる ない', 'VISAの勘定科目がわからない'),
 (0.18142909608410532,
  '海外 出張 する 旅費 精算 する どう する 良い',
  '海外出張したときの旅費精算をしたいがどうしたら良い'),
 (0.17527067146848016, '育児 短時間 勤務 申請 書 曜日 別 申請 できる', '育児短時間勤務申請書は曜日別に申請できるか？'),
 (0.17417897115045625, '育児 短時間 勤務 申請 どこ 提出 する よい', '育児短時間勤務申請はどこへ提出すればよいか？'),
 (0.17393023556911802, '育児 短時間 勤務 申請 書 いつ 提出 する', '育児短時間勤務申請書はいつまでに提出するのか？')]
```


## 子供がうまれた

既存

```
[(0.70710678118654757, '子供 生まれる', '子供が生まれた'),
 (0.0, '育児 短時間 勤務 申請 書 いつ 提出 する', '育児短時間勤務申請書はいつまでに提出するのか？'),
 (0.0, '育児 短時間 勤務 申請 書 曜日 別 申請 できる', '育児短時間勤務申請書は曜日別に申請できるか？'),
 (0.0, '育児 短時間 勤務 申請 どこ 提出 する よい', '育児短時間勤務申請はどこへ提出すればよいか？'),
 (0.0, 'ＶＩＳＡ 勘定 科目 わかる ない', 'VISAの勘定科目がわからない'),
 (0.0, '海外 出張 する 旅費 精算 する どう する 良い', '海外出張したときの旅費精算をしたいがどうしたら良い'),
 (0.0, '保険証 失 くい', '保険証を失くした')]
```

LSI

```
[(1.0, '子供 生まれる', '子供が生まれた'),
 (1.3972112760024048e-16,
  '育児 短時間 勤務 申請 書 曜日 別 申請 できる',
  '育児短時間勤務申請書は曜日別に申請できるか？'),
 (1.1102230246251565e-16, 'ＶＩＳＡ 勘定 科目 わかる ない', 'VISAの勘定科目がわからない'),
 (5.5511151231257827e-17, '保険証 失 くい', '保険証を失くした'),
 (1.4328446680903843e-17,
  '育児 短時間 勤務 申請 書 いつ 提出 する',
  '育児短時間勤務申請書はいつまでに提出するのか？'),
 (-5.3945024184152252e-17,
  '育児 短時間 勤務 申請 どこ 提出 する よい',
  '育児短時間勤務申請はどこへ提出すればよいか？'),
 (-1.3981097219342767e-16,
  '海外 出張 する 旅費 精算 する どう する 良い',
  '海外出張したときの旅費精算をしたいがどうしたら良い')]
```

LDA

```
[(0.99722593860453557, '子供 生まれる', '子供が生まれた'),
 (0.98530465203867224, '育児 短時間 勤務 申請 書 曜日 別 申請 できる', '育児短時間勤務申請書は曜日別に申請できるか？'),
 (0.98498486954174702, '育児 短時間 勤務 申請 どこ 提出 する よい', '育児短時間勤務申請はどこへ提出すればよいか？'),
 (0.98491433030420661, '育児 短時間 勤務 申請 書 いつ 提出 する', '育児短時間勤務申請書はいつまでに提出するのか？'),
 (0.21122679367587543, '保険証 失 くい', '保険証を失くした'),
 (0.18968996274253319, 'ＶＩＳＡ 勘定 科目 わかる ない', 'VISAの勘定科目がわからない'),
 (0.18146405542426788,
  '海外 出張 する 旅費 精算 する どう する 良い',
  '海外出張したときの旅費精算をしたいがどうしたら良い')]
```

## わからない

既存

```
[(0.63245553203367599, 'ＶＩＳＡ 勘定 科目 わかる ない', 'VISAの勘定科目がわからない'),
 (0.0, '育児 短時間 勤務 申請 書 いつ 提出 する', '育児短時間勤務申請書はいつまでに提出するのか？'),
 (0.0, '育児 短時間 勤務 申請 書 曜日 別 申請 できる', '育児短時間勤務申請書は曜日別に申請できるか？'),
 (0.0, '育児 短時間 勤務 申請 どこ 提出 する よい', '育児短時間勤務申請はどこへ提出すればよいか？'),
 (0.0, '子供 生まれる', '子供が生まれた'),
 (0.0, '海外 出張 する 旅費 精算 する どう する 良い', '海外出張したときの旅費精算をしたいがどうしたら良い'),
 (0.0, '保険証 失 くい', '保険証を失くした')]
```

LSI

```
[(1.0, 'ＶＩＳＡ 勘定 科目 わかる ない', 'VISAの勘定科目がわからない'),
 (7.208598777808593e-17,
  '育児 短時間 勤務 申請 書 曜日 別 申請 できる',
  '育児短時間勤務申請書は曜日別に申請できるか？'),
 (2.8164686957795376e-17,
  '育児 短時間 勤務 申請 書 いつ 提出 する',
  '育児短時間勤務申請書はいつまでに提出するのか？'),
 (-9.2346576663475958e-18,
  '海外 出張 する 旅費 精算 する どう する 良い',
  '海外出張したときの旅費精算をしたいがどうしたら良い'),
 (-2.7755575615628914e-17, '子供 生まれる', '子供が生まれた'),
 (-7.5839904124817604e-17,
  '育児 短時間 勤務 申請 どこ 提出 する よい',
  '育児短時間勤務申請はどこへ提出すればよいか？'),
 (-1.1102230246251565e-16, '保険証 失 くい', '保険証を失くした')]
```

LDA

```
[(0.99736341148713947, 'ＶＩＳＡ 勘定 科目 わかる ない', 'VISAの勘定科目がわからない'),
 (0.99624312128717407,
  '海外 出張 する 旅費 精算 する どう する 良い',
  '海外出張したときの旅費精算をしたいがどうしたら良い'),
 (0.19334402464480621, '子供 生まれる', '子供が生まれた'),
 (0.17477051676111335, '保険証 失 くい', '保険証を失くした'),
 (0.1419271106930749, '育児 短時間 勤務 申請 書 曜日 別 申請 できる', '育児短時間勤務申請書は曜日別に申請できるか？'),
 (0.14093549942089204, '育児 短時間 勤務 申請 どこ 提出 する よい', '育児短時間勤務申請はどこへ提出すればよいか？'),
 (0.14071763783245683, '育児 短時間 勤務 申請 書 いつ 提出 する', '育児短時間勤務申請書はいつまでに提出するのか？')]
```
